In [1]:
# import important packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import sys
sys.path.insert(0,'../scripts/')
import warnings
warnings.filterwarnings(action="ignore")

In [2]:
df = pd.read_csv("../data/AdSmartABdata.csv")
df_control= pd.read_csv("../data/control_group_data.csv")
df_exposed= pd.read_csv("../data/exposed_clean_data.csv")